In [ ]:
import torch
import sys
sys.path.append('/work/jprieto/source/Keratoconus/src/py')
import dataset
import argparse
import nets 
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd


In [2]:
parser = argparse.ArgumentParser(description='Train a model on the keratoconus dataset')

parser = dataset.ImgSegDataModule.add_data_specific_args(parser)

In [3]:
from argparse import Namespace
def get_argparse_dict(parser):
    # Get the default arguments from the parser
    default = {}
    for action in parser._actions:
        if action.dest != "help":
            default[action.dest] = action.default
    return default

args = get_argparse_dict(parser)

args = Namespace(**args)

In [ ]:

# args_d = vars(args)

# args_d['csv_train'] = '/CMF/data/lumargot/Keratoconus/preprocess/kc_filtered_trachoma_seg_train_train.csv'
# args_d['csv_valid'] = '/CMF/data/lumargot/Keratoconus/preprocess/kc_filtered_trachoma_seg_train_test.csv'
# args_d['csv_test'] = '/CMF/data/lumargot/Keratoconus/preprocess/kc_filtered_trachoma_seg_test.csv'
# args_d['mount_point'] = '/CMF/data/lumargot/Keratoconus/preprocess'

# dm = dataset.ImgSegDataModule(**args_d)
# dm.setup()

# train_dl = dm.train_dataloader()

In [4]:

df = pd.read_csv('/CMF/data/lumargot/Keratoconus/csv/keratoconous_files_seg.csv')


In [ ]:
# px.histogram(df, x="K1", nbins=4)

In [ ]:
# px.histogram(df, x="K2", nbins=4)

In [ ]:
# px.histogram(df, x="Kmean", nbins=4)

In [5]:
test_ds = dataset.ImgSegDataset(df, mount_point='/CMF/data/lumargot/Keratoconus/preprocess/', img_column='path', seg_column='seg', transform=dataset.EvalTransform(img_key="path", seg_key="seg"))

In [ ]:
model = nets.AutoEncoderKL.load_from_checkpoint('/CMF/data/lumargot/Keratoconus/train_output/autoencoderkl/v0.2/epoch=149-val_loss=0.02.ckpt')

In [7]:
model = model.eval().cuda()
z_mus = []
with torch.no_grad():

    for img_d in test_ds:
    
        img = img_d['path']
        
        _, z_mu, z_sigma = model(img.unsqueeze(0).cuda())
        z_mus.append(z_mu)



In [8]:
z_mus_np = torch.cat(z_mus, dim=0).flatten(1).cpu().numpy()

In [ ]:



# df['class'] = pd.cut(df['K1'], bins=4, labels=[0, 1, 2, 3])
# px.histogram(df, x="class", nbins=4)

# Define the bins and labels
bins = [-float('inf'), 40, 47, float('inf')]
labels = [0, 1, 2]
df['class'] = pd.cut(df['K1'], bins=bins, labels=labels, right=False)

px.histogram(df, x="class")

In [52]:
from sklearn.model_selection import train_test_split

X = z_mus_np
y = df['class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)



In [ ]:
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import numpy as np

# Create histogram traces for y_train and y_test
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=y_train,
    name='y_train',
    marker=dict(color='blue', opacity=0.7)
))

fig.add_trace(go.Histogram(
    x=y_test,
    name='y_test',
    marker=dict(color='orange', opacity=0.7)
))

# Update layout for stacked histogram
fig.update_layout(
    barmode='stack',
    title='Stacked Histogram of y_train and y_test',
    xaxis_title='Values',
    yaxis_title='Count',
    legend_title='Dataset',
    template='plotly_white'
)

# Show the plot
fig.show()


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier#create new a knn model
import numpy as np


knn = KNeighborsClassifier()#create a dictionary of all values we want to test for n_neighbors
params_knn = {'n_neighbors': np.arange(1, 25)}#use gridsearch to test all values for n_neighbors
knn_gs = GridSearchCV(knn, params_knn, cv=5)#fit model to training data
knn_gs.fit(X_train, y_train)

In [ ]:
#save best model
knn_best = knn_gs.best_estimator_#check best n_neigbors value
print(knn_gs.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestClassifier#create a new random forest classifier
rf = RandomForestClassifier()#create a dictionary of all values we want to test for n_estimators
params_rf = {'n_estimators': [5, 10, 50, 100, 200, 1000]}#use gridsearch to test all values for n_estimators
rf_gs = GridSearchCV(rf, params_rf, cv=5)#fit model to training data
rf_gs.fit(X_train, y_train)

In [ ]:
#save best model
rf_best = rf_gs.best_estimator_#check best n_estimators value
print(rf_gs.best_params_)

In [ ]:
from sklearn.linear_model import LogisticRegression#create a new logistic regression model
log_reg = LogisticRegression(max_iter=10000)#fit the model to the training data
log_reg.fit(X_train, y_train)

In [ ]:
#test the three models with the test data and print their accuracy scores

print('knn: {}'.format(knn_best.score(X_test, y_test)))
print('rf: {}'.format(rf_best.score(X_test, y_test)))
print('log_reg: {}'.format(log_reg.score(X_test, y_test)))

In [60]:
from sklearn.ensemble import VotingClassifier#create a dictionary of our models
estimators=[('knn', knn_best), ('rf', rf_best), ('log_reg', log_reg)]#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='hard')

In [ ]:
#fit model to training data
ensemble.fit(X_train, y_train)#test our model on the test data
ensemble.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


X_pred = ensemble.predict(X_test)

# Assuming y_test contains true labels and X_pred contains predicted labels
conf_matrix = confusion_matrix(y_test, X_pred, normalize='true')

# Create a confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix)

# Plot the confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


X_pred = log_reg.predict(X_test)

# Assuming y_test contains true labels and X_pred contains predicted labels
conf_matrix = confusion_matrix(y_test, X_pred, normalize='true')

# Create a confusion matrix display
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=ensemble.classes_)

# Plot the confusion matrix
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

In [ ]:
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_test, X_pred)

# Print the report
print("Classification Report:")
print(report)